In [5]:
import polars as pl
import numpy as np
import requests
from zipfile import ZipFile

In [2]:
# Download file anew: (takes 13 m 12s)
chunk_size = 4096
filename = "C:/Users/Mette/OneDrive - Danmarks Tekniske Universitet/11. Semester Speciale/Data/Xena/TCGA_PANCAN_transcript_expression_RNAseq_RSEM_tpm.gz"
document_url = "https://toil-xena-hub.s3.us-east-1.amazonaws.com/download/tcga_rsem_isoform_tpm.gz"
#with requests.get(document_url, stream=True) as r:
#        with open(filename, 'wb') as f:
#            for chunk in r.iter_content(chunk_size): 
#                if chunk:
#                    f.write(chunk)

In [3]:
in_file = "C:/Users/Mette/OneDrive - Danmarks Tekniske Universitet/11. Semester Speciale/Data/Xena/TCGA_PANCAN_transcript_expression_RNAseq_RSEM_tpm.gz"
in_pheno2 = "C:/Users/Mette/OneDrive - Danmarks Tekniske Universitet/11. Semester Speciale/Data/Xena/TCGA_PANCAN_curated_clinical_data.tsv"
in_transcript_id = "C:/Users/Mette/OneDrive - Danmarks Tekniske Universitet/11. Semester Speciale/Data/Xena/TCGA_PANCAN_transcript_expression_RNAseq_RSEM_transcript_list.txt"
in_genenames = "C:/Users/Mette/OneDrive - Danmarks Tekniske Universitet/11. Semester Speciale/Data/targets_ensg.csv"

In [6]:
# Load dataframe as lazy using polars
lf_data = pl.scan_csv(in_file, separator="\t")

# Load transcript ids:
with open(in_transcript_id) as f:  
    transcript_id_list = f.read().splitlines()
transcript_id_list.pop(0) # Remove "column name"s

# Add it to the lf_data: 
lf_data = lf_data.with_columns(trans_id = transcript_id_list)

# Load gene names as dataframe: 
df_genenames = pl.read_csv(in_genenames)

In [7]:
len(lf_data.collect_schema().names())

ComputeError: out of memory

This error occurred with the following context stack:
	[1] 'csv scan' failed
	[2] 'with_columns' input failed to resolve


In [5]:
# Make list of sample ids for different cancer types
df_pheno = pl.read_csv(in_pheno2, separator="\t")
sample_id_PAAD = df_pheno.filter(pl.col("cancer type abbreviation") == "PAAD").select(pl.col("sample")).to_numpy()
sample_id_data = np.array(lf_data.collect_schema().names())
sample_id_PAAD = set(np.intersect1d(sample_id_PAAD, sample_id_data))

# Subset lazyframe to include only samples from cancer type of interest.
#lf_data_PAAD = lf_data.select(sample_id_PAAD.union(set(["trans_id"])))

In [5]:
# From ensembl API get the transcript IDs for all genes of interest: 

# Define a function to fetch transcript IDs from Ensembl
def fetch_transcript_ids_from_ensembl(gene_id):
    url = f"https://rest.ensembl.org/lookup/id/{gene_id}?expand=1"
    response = requests.get(url, headers={"Content-Type": "application/json"})
    if response.ok:
        data = response.json()
        return [transcript['id'] for transcript in data.get('Transcript', [])]
    else:
        return []

# Apply the API function to the DataFrame
df_genenames = df_genenames.with_columns(pl.col("converted_alias").map_elements(fetch_transcript_ids_from_ensembl).alias("transcript_ids"))

# Get a list of all the transcript IDs related to any of the 14 genes of interest: 
trans_id_of_interest = df_genenames.select(pl.col("transcript_ids").explode()).to_series().to_list()


C:\Users\Mette\AppData\Local\Temp\ipykernel_14260\1198829557.py:14: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  df_genenames = df_genenames.with_columns(pl.col("converted_alias").map_elements(fetch_transcript_ids_from_ensembl).alias("transcript_ids"))


In [1]:
lf_data_PAAD_subset

NameError: name 'lf_data_PAAD_subset' is not defined

In [7]:
#lf_data_PAAD.filter(pl.col("trans_id").is_in(pl.Series(trans_id_of_interest)))

lf_trans_id_of_interest = pl.LazyFrame({"trans_id": trans_id_of_interest})
lf_data_PAAD_subset = lf_data_PAAD.join(lf_trans_id_of_interest, on="trans_id", how="inner")

sample_lf = lf_data_PAAD_subset.limit(5)  # Limit to first 1000 rows for testing
df_sample = sample_lf.collect()
print(df_sample)

: 

In [56]:
set(["trans_id"]).union(sample_id_PAAD)

sample_id_PAAD.union(set(["trans_id"]))

{'TCGA-2J-AAB1-01',
 'TCGA-2J-AAB4-01',
 'TCGA-2J-AAB6-01',
 'TCGA-2J-AAB8-01',
 'TCGA-2J-AAB9-01',
 'TCGA-2J-AABA-01',
 'TCGA-2J-AABE-01',
 'TCGA-2J-AABF-01',
 'TCGA-2J-AABH-01',
 'TCGA-2J-AABI-01',
 'TCGA-2J-AABK-01',
 'TCGA-2J-AABO-01',
 'TCGA-2J-AABP-01',
 'TCGA-2J-AABR-01',
 'TCGA-2J-AABT-01',
 'TCGA-2J-AABU-01',
 'TCGA-2J-AABV-01',
 'TCGA-2L-AAQA-01',
 'TCGA-2L-AAQE-01',
 'TCGA-2L-AAQI-01',
 'TCGA-2L-AAQJ-01',
 'TCGA-2L-AAQL-01',
 'TCGA-2L-AAQM-01',
 'TCGA-3A-A9I5-01',
 'TCGA-3A-A9I7-01',
 'TCGA-3A-A9I9-01',
 'TCGA-3A-A9IB-01',
 'TCGA-3A-A9IC-01',
 'TCGA-3A-A9IH-01',
 'TCGA-3A-A9IJ-01',
 'TCGA-3A-A9IL-01',
 'TCGA-3A-A9IN-01',
 'TCGA-3A-A9IO-01',
 'TCGA-3A-A9IR-01',
 'TCGA-3A-A9IS-01',
 'TCGA-3A-A9IU-01',
 'TCGA-3A-A9IV-01',
 'TCGA-3A-A9IX-01',
 'TCGA-3A-A9IZ-01',
 'TCGA-3A-A9J0-01',
 'TCGA-3E-AAAY-01',
 'TCGA-3E-AAAZ-01',
 'TCGA-F2-6879-01',
 'TCGA-F2-6880-01',
 'TCGA-F2-7273-01',
 'TCGA-F2-7276-01',
 'TCGA-F2-A44G-01',
 'TCGA-F2-A44H-01',
 'TCGA-F2-A7TX-01',
 'TCGA-F2-A8YN-01',
